#### Part 1: Read cell body position from .apo files, downsample and save them as txt file for applying inverse transformation

In [70]:
import Neuron_analysis as na
from Neuron_analysis import *

import os
import pandas as pd
import re

import tkinter as tk
import tkinter.filedialog as fdialog
from tkinter import simpledialog

import skimage

import numpy as np
import matplotlib.pyplot as plt

from read_roi import read_roi_file

In [68]:
indir = fdialog.askdirectory(title='Select the directories with all the cell bodies.apo')
outdir = fdialog.askdirectory(title='Select the output directory to store your file')

files = os.listdir(indir)

In [69]:
def prepare_cellbody(indir,anno_file,outdir):
    anno=open(os.path.join(indir,anno_file),'r')
    anno_data=anno.readlines()
 
    headings=anno_data[0].rstrip('\n').split(',')
    annotations=[lines.rstrip('\n').split(',') for lines in anno_data[1:]]
    annotation_df=pd.DataFrame(annotations, columns=headings)
    
    cell_body= annotation_df[annotation_df.color_g=='255']
    #pickout only green points!
    
    # find the coordinates and downsample it! these values are based on xy= 0.6 z= 5, and downsampling to 25 xyz
    ds_x= pd.to_numeric(cell_body['x'])//41
    ds_y= pd.to_numeric(cell_body['y'])//41
    ds_z= pd.to_numeric(cell_body['z'])//5

    x=ds_x.astype(int).astype(str)
    y=ds_y.astype(int).astype(str)
    z=ds_z.astype(int).astype(str)
    
    m=re.search('\D{2}[0-9]{3}', anno_file)
    mouse_name=m[0]

    out_name=os.path.join(outdir,mouse_name+'cellbody.txt')

    f=open(out_name,'w+')
    f.write('point'+'\n')
    f.write('1'+'\n') # by default there is only 1 cell body, hence only 1 point here
    f.write(x.values[0]+ ' ' + y.values[0]+ ' '+ z.values[0])
    f.close()
    return
    

In [70]:
for items in files:
    prepare_cellbody(indir, items, outdir)

### Part2  : (after transformation by transformix) extract the transformed points, refill y section (because we cropped the template for registration). 
Then write as a single file containing brain ID and cell body location

___Work in progress___

In [24]:
brain_id= ['AL175', 'AL223','AL230', 'AL236', 'AL244', 'AL261']
#missing 'AL225' and 'AL242' for now
cb_file=f'D:\{brain_id[1]}\points\cellbody\outputpoints.txt'

In [60]:
all_points=[]

for items in brain_id:
    this_file=cb_file=f'D:\{items}\points\cellbody\outputpoints.txt'
    
    template_file = [x for x in os.listdir(f'D:\{items}\\') if 'template' in x]
    to_add=na.find_crop(template_file[0])
  
    with open(cb_file,'r') as output:
        outputpoint= output.readlines()

    for lines in outputpoint:
        m=re.search("(?:OutputIndexFixed = \[ )([0-9]+ [0-9]+ [0-9]+)", lines).groups(0)
        this_line= str(m[0]).split(' ')
        mypoints= [int(stuff) for stuff in this_line]
        mypoints[1]= mypoints[1]+to_add
        mypoints.insert(0, items)
        all_points.append(mypoints)
        
# the coordiates are arranged in x,y and z

In [59]:
all_points

[['AL175', 92, 218, 52],
 ['AL223', 99, 214, 47],
 ['AL230', 88, 210, 56],
 ['AL236', 104, 214, 40],
 ['AL244', 86, 237, 47],
 ['AL261', 123, 196, 42]]

In [61]:
all_points

[['AL175', 92, 283, 52],
 ['AL223', 99, 279, 47],
 ['AL230', 88, 275, 56],
 ['AL236', 104, 279, 40],
 ['AL244', 86, 302, 47],
 ['AL261', 123, 261, 42]]

In [50]:
to_add= na.find_crop(template_file[0])

65

In [51]:
mypoints

['AL261', 123, 196, 42]

## Part for viral cellbody ROIs (after segmentation and curation in LabKit)

ImageJ plugin https://imagej.net/plugins/labkit/

In [95]:
outdir = fdialog.askdirectory(title='Select the output directory to store your file')

In [73]:
image_name = fdialog.askopenfile(title='Select the binary image').name
mouse_name=na.find_mousename(image_name)


In [107]:
def prepare_cellbody_virus(tiff_file, threshold=5):
    '''From curated segmentation tiff file, find connected components(3d) with their area and centeroid.
    
    threshold: connected components with numbers of pixels below this number will get removed
    
    saves .txt file compatible for transformix (and downsampled to 25um space) and a .csv file listing the area and the centroid x,y,z coordinate (not downsampled) for each component
    '''
    
    im= skimage.io.imread(tiff_file) 
        
    labels=skimage.measure.label(im)
    info= skimage.measure.regionprops(labels)
    
    real_cells= [items for items in info if items.area>threshold]
    # remove connected components that are less than 5 pixels for example
    areas=[item.area for item in real_cells]
    
    center= [item.centroid for item in real_cells]
    center=np.asarray(center)
    
    headings=['x', 'y', 'z']
    #points=[lines.rstrip('\n').split(',') for lines in anno_data[1:]]
    centroid_df=pd.DataFrame(center, columns=headings)
    centroid_df['area']= areas
    return centroid_df

def refill_roi(centroid_df):
    
    ''' Find x, y, z section to refill from .roi file (account for x and y) and substack file name (account for z)
    refill the input dataframe's 'x', 'y', 'z' to match the original image size. Input data frame is usually an output from prepare_cellbody_virus
    '''
    roi_name = fdialog.askopenfile(title='Select the roi').name
    roi = read_roi_file(roi_name)
    name=list(roi.keys())[0]
    x_add=roi[name]['left']-1
    y_add=roi[name]['top']-1
    substack_name = fdialog.askopenfile(title='Select the substack used for cellbody segmentation').name
    m= re.search("(?<=_)[0-9]{1,3}.[0-9]{1,3}",substack_name)[0]
    to_add= m.split('-')
    lead= int(to_add[0])-1
    centroid_df['x']+= x_add
    centroid_df['y']+= y_add
    centroid_df['z']+= lead
    
    return centroid_df

def make_transformix_txt(centroid_df,mouse_name): #work in progress

    # find the coordinates and downsample it! these values are based on xy= 0.6 z= 5, and downsampling to 25 xyz
    ds_x= pd.to_numeric(centroid_df['x'])//5
    ds_y= pd.to_numeric(centroid_df['y'])//5
    ds_z= pd.to_numeric(centroid_df['z'])//5

    x=ds_x.astype(int).astype(str)
    y=ds_y.astype(int).astype(str)
    z=ds_z.astype(int).astype(str)
    
    q = [' '.join(x) for x in zip(x,y,z)]
    num_row=len(q)

    out_name=os.path.join(outdir,mouse_name+'cellbody.txt')

    f=open(out_name,'w+')
    f.write('point'+'\n')
    f.write(str(num_row)+'\n')
    for lines in q:
        f.write(lines+'\n')
    f.close()
    
    return

In [91]:
cell_body=prepare_cellbody_virus(image_name, threshold=5)
cell_body

,x,y,z,area
0,3.703704,180.407407,153.518519,27
1,7.030075,119.902256,150.300752,133
2,17.102345,137.281450,133.098081,469
3,14.987342,120.278481,143.550633,158
4,13.380952,121.492063,201.539683,63
...,...,...,...,...
176,128.333333,59.666667,43.875000,24
177,128.869565,80.630435,36.760870,46
178,132.437500,65.687500,47.875000,16
179,134.555556,74.888889,21.222222,9


In [92]:
cell_body_refill= refill_roi(cell_body)
cell_body_refill

In [98]:
out_name=os.path.join(outdir,mouse_name+'cellbody.csv')
cell_body_refill.to_csv(out_name)

In [108]:
make_transformix_txt(cell_body_refill,mouse_name)

In [99]:
cell_body_refill

,x,y,z,area
0,448.703704,1059.407407,402.518519,27
1,452.030075,998.902256,399.300752,133
2,462.102345,1016.281450,382.098081,469
3,459.987342,999.278481,392.550633,158
4,458.380952,1000.492063,450.539683,63
...,...,...,...,...
176,573.333333,938.666667,292.875000,24
177,573.869565,959.630435,285.760870,46
178,577.437500,944.687500,296.875000,16
179,579.555556,953.888889,270.222222,9


249